In [1]:
import pandas as pd
from matplotlib import pyplot as plt
import numpy as np


In [2]:
import warnings
warnings.filterwarnings('ignore')

In [3]:
df = pd.read_csv("customer_churn.csv")
df

FileNotFoundError: [Errno 2] No such file or directory: 'customer_churn.csv'

In [ ]:
df.Churn.value_counts()

In [ ]:
df.drop('customerID',axis='columns',inplace=True)

In [ ]:
df.dtypes

In [ ]:
pd.to_numeric(df.TotalCharges,errors='coerce').isna()

In [ ]:
df[pd.to_numeric(df.TotalCharges,errors='coerce').isna()]

In [ ]:
df.shape

In [ ]:
df.dropna()

In [ ]:
df.shape

In [ ]:
df[df.TotalCharges!=' '].shape

In [ ]:
df1 = df[df.TotalCharges!=' ']
df1.shape

In [ ]:
df1.dtypes

In [ ]:
df1.TotalCharges = pd.to_numeric(df1.TotalCharges)

In [ ]:
df1.TotalCharges.values

In [ ]:
df1[df1.Churn=='No']

**Data Visualization**

In [ ]:
no_churnTenure = df1[df1.Churn=='No'].tenure
yes_churnTenure = df1[df1.Churn=='Yes'].tenure

plt.xlabel("tenure")
plt.ylabel("Customers")
plt.title("Churn Prediction for customer")

plt.hist([yes_churnTenure, no_churnTenure], rwidth=0.95, color=['black','blue'],label=['Churn=Yes','Churn=No'])
plt.legend()

In [ ]:
no_MonthlyCharges = df1[df1.Churn=='No'].MonthlyCharges      
yes_MonthlyCharges = df1[df1.Churn=='Yes'].MonthlyCharges      

plt.xlabel("Monthly Charges")
plt.ylabel("Customers")
plt.title("Customer Churn Prediction(Monthly)")

plt.hist([yes_MonthlyCharges, no_MonthlyCharges], rwidth=0.95, color=['yellow','green'],label=['Churn=Yes','Churn=No'])
plt.legend()

In [ ]:
def print_unique_col_values(df):
       for column in df:
            if df[column].dtypes=='object':
                print(f'{column}: {df[column].unique()}') 

In [ ]:
print_unique_col_values(df1)

In [ ]:
df1.replace('No internet service','No',inplace=True)
df1.replace('No phone service','No',inplace=True)

In [ ]:
print_unique_col_values(df1)

In [ ]:
columnsY_N= ['Partner','Dependents','PhoneService','MultipleLines','OnlineSecurity','OnlineBackup',
                  'DeviceProtection','TechSupport','StreamingTV','StreamingMovies','PaperlessBilling','Churn']
for col in columnsY_N:
    df1[col].replace({'Yes': 1,'No': 0},inplace=True)

In [ ]:
for col in df1:
    print(f'{col}: {df1[col].unique()}') 

In [ ]:
df1['gender'].replace({'Female':1,'Male':0},inplace=True)

In [ ]:
df1.gender.unique()

In [ ]:
df2 = pd.get_dummies(data=df1, columns=['InternetService','Contract','PaymentMethod'])
df2.columns

In [ ]:
df2.sample(5)

In [ ]:
df2.dtypes

In [ ]:
Scaler = ['tenure','MonthlyCharges','TotalCharges']

from sklearn.preprocessing import MinMaxScaler
scaler = MinMaxScaler()
df2[Scaler] = scaler.fit_transform(df2[Scaler])

In [ ]:
for col in df2:
    print(f'{col}: {df2[col].unique()}')

In [ ]:
X = df2.drop('Churn',axis='columns')
y = testLabels = df2.Churn.astype(np.float32)

from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=15, stratify=y)

In [ ]:
y_train.value_counts()

In [ ]:
y.value_counts()

In [ ]:
y_test.value_counts()

In [ ]:
X_train.shape

In [ ]:
X_test.shape

In [ ]:
X_train[:10]

In [ ]:
len(X_train.columns)

# Use logistic regression (task)

In [ ]:
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import classification_report

In [ ]:
def logistic(X_train, y_train, X_test, y_test, weights):
    if weights==-1:
        model = LogisticRegression()
    else:
        model = LogisticRegression(class_weight={0:weights[0], 1:weights[1]})

    model.fit(X_train, y_train)
    acc = model.score(X_test, y_test)
    print("Accuracy", acc, "\n")

    y_pred = model.predict(X_test)
    print("preds", y_pred[:5], "\n")

    cl_rep = classification_report(y_test,y_pred)
    print(cl_rep)

In [ ]:
weights = -1 
logistic(X_train, y_train, X_test, y_test, weights)

In [ ]:
weights = [1, 1.5]
logistic(X_train, y_train, X_test, y_test, weights)

## Undersampling

In [ ]:
count_class_0, count_class_1 = df1.Churn.value_counts()

df_class_0 = df2[df2['Churn'] == 0]
df_class_1 = df2[df2['Churn'] == 1]

In [ ]:
df_class_0_under = df_class_0.sample(count_class_1)
df_test_under = pd.concat([df_class_0_under, df_class_1], axis=0)

print('Random UnderSampling:')
print(df_test_under.Churn.value_counts())

In [ ]:
X = df_test_under.drop('Churn',axis='columns')
y = df_test_under['Churn']

from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=15, stratify=y)

In [ ]:
# Number of classes in training Data
y_train.value_counts()

#### Applying Logistic Regression

In [ ]:
weights = -1 # pass -1 to use Logistics Regression without weights
logistic(X_train, y_train, X_test, y_test, weights)

### ==============================================================
###  Oversampling

In [ ]:
# Oversample 1-class and concat the DataFrames of both classes
df_class_1_over = df_class_1.sample(count_class_0, replace=True)
df_test_over = pd.concat([df_class_0, df_class_1_over], axis=0)

print('Random over-sampling:')
print(df_test_over.Churn.value_counts())

In [ ]:
X = df_test_over.drop('Churn',axis='columns')
y = df_test_over['Churn']

from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=15, stratify=y)

In [ ]:
# Number of classes in training Data
y_train.value_counts()

#### Logistic Regression

In [ ]:
weights = -1 # pass -1 to use Logistics Regression without weights
logistic(X_train, y_train, X_test, y_test, weights)

### ===================================================
### SMOTE   =====>>> Must install "pip install imbalanced-learn 

In [ ]:
X = df2.drop('Churn',axis='columns')
y = df2['Churn']

In [ ]:
from imblearn.over_sampling import SMOTE


smote = SMOTE(sampling_strategy='minority')
# use fit_resample()
X_sm, y_sm = smote.fit_resample(X, y)

y_sm.value_counts()

In [ ]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X_sm, y_sm, test_size=0.2, random_state=15, stratify=y_sm)

In [ ]:
# Number of classes in training Data
y_train.value_counts()

#### Logistic Regression

In [ ]:
weights = -1 # pass -1 to use Logistics Regression without weights
logistic(X_train, y_train, X_test, y_test, weights)

### =======================================================
###   Ensemble

In [ ]:
df2.Churn.value_counts()

In [ ]:
# Regain Original features and labels
X = df2.drop('Churn',axis='columns')
y = df2['Churn']

In [ ]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=15, stratify=y)

In [ ]:
y_train.value_counts()

### increase the minorty class with equal to his nubmer

In [ ]:
model = LogisticRegression()

df3 = X_train.copy()
df3['Churn'] = y_train

In [ ]:
df3.head()

In [ ]:
df3_class0 = df3[df3.Churn==0]
df3_class1 = df3[df3.Churn==1]

In [ ]:
def get_train_batch(df_majority, df_minority, start, end):
    df_train = pd.concat([df_majority[start:end], df_minority], axis=0)
    X_train = df_train.drop('Churn', axis='columns')
    y_train = df_train.Churn
    return X_train, y_train    

In [ ]:
X_train, y_train = get_train_batch(df3_class0, df3_class1, 0, 1495)

model1 = LogisticRegression()
model1.fit(X_train, y_train)
y_pred1 = model1.predict(X_test)

In [ ]:
X_train, y_train = get_train_batch(df3_class0, df3_class1, 1495, 2990)

model2 = LogisticRegression()
model2.fit(X_train, y_train)
y_pred2 = model2.predict(X_test)

In [ ]:
X_train, y_train = get_train_batch(df3_class0, df3_class1, 2990, 4130)

model3 = LogisticRegression()
model3.fit(X_train, y_train)
y_pred3 = model3.predict(X_test)

In [ ]:
len(y_pred1)

In [ ]:
y_pred_final = y_pred1.copy()
for i in range(len(y_pred1)):
    n_ones = y_pred1[i] + y_pred2[i] + y_pred3[i]
    if n_ones>1:
        y_pred_final[i] = 1
    else:
        y_pred_final[i] = 0

In [ ]:
cl_rep = classification_report(y_test, y_pred_final)
print(cl_rep)